In [1]:
pip install yfinance statsforecast matplotlib pandas


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import yfinance as yf
import pandas as pd
from statsforecast import StatsForecast
from statsforecast.models import AutoETS
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib.pyplot as plt

symbols = [
    'GFNORTEO.MX', 'AMXL.MX', 'WALMEX.MX', 'GMEXICOB.MX', 'CEMEXCPO.MX',
    'KOF.MX', 'PE&OLES.MX', 'BIMBOA.MX', 'ELEKTRA.MX', 'GAPB.MX', 
    'TLEVISACPO.MX', 'ALSEA.MX', 'OMAB.MX', 'AC.MX', 'GRUMAB.MX',
    'FEMSAUBD.MX', 'LABB.MX', 'GFINBURO.MX', 'LIVEPOLC-1.MX', 'ICHB.MX',
    'CUERVO.MX', 'BSMXB.MX', 'PINFRA.MX', 'SANMEXB.MX', 'MEGACPO.MX',
    'IENOVA.MX', 'ASURB.MX', 'GMXT.MX', 'GENTERA.MX', 'RA.MX'
]

data = yf.download(symbols, start="2015-01-01", end="2024-01-01", interval='1mo')['Adj Close']

data_long = data.reset_index().melt(id_vars="Date", var_name="unique_id", value_name="y")
data_long = data_long.rename(columns={"Date": "ds"}).dropna()

print(data_long.head())


In [ ]:
train_data = data_long[data_long['ds'] < '2023-10-01']
test_data = data_long[data_long['ds'] >= '2023-10-01']


In [ ]:
model = StatsForecast(df=train_data, models=[AutoETS()], freq='M')
model.fit()

forecast_horizon = len(test_data['ds'].unique())
forecasts = model.predict(h=forecast_horizon)

merged_data = test_data.merge(forecasts, on=['unique_id', 'ds'])
print(merged_data.head())


In [ ]:
errors = merged_data.groupby('unique_id').apply(
    lambda df: mean_absolute_percentage_error(df['y'], df['AutoETS'])
).reset_index(name='MAPE')

best_fit = errors.nsmallest(1, 'MAPE')
worst_fit = errors.nlargest(1, 'MAPE')
print("Mejor ajuste:", best_fit)
print("Peor ajuste:", worst_fit)


In [ ]:
best_id = best_fit['unique_id'].values[0]
worst_id = worst_fit['unique_id'].values[0]

def plot_series(series_id, title):
    series_data = merged_data[merged_data['unique_id'] == series_id]
    plt.figure(figsize=(10, 6))
    plt.plot(series_data['ds'], series_data['y'], label='Datos reales')
    plt.plot(series_data['ds'], series_data['AutoETS'], label='Pronóstico', color='red')
    plt.title(f"{title} (MAPE: {series_data['MAPE'].values[0]:.2f})")
    plt.xlabel('Fecha')
    plt.ylabel('Precio')
    plt.legend()
    plt.show()

plot_series(best_id, 'Serie con el mejor ajuste')
plot_series(worst_id, 'Serie con el peor ajuste')
